In [ ]:
import re

import parse
import pdfplumber
import pandas as pd
from collections import namedtuple

pd.set_option('display.max_rows', None)  # or 1000


import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
import os

# Change the current working directory to H:/My Drive
os.chdir('H:/Meu Drive/CDT - MÉDIAS 2023/')

In [ ]:
if os.path.isdir('H:/Meu Drive/CDT - MÉDIAS 2023/1_ANO/'):
    print('Directory exists')
else:
    print('Directory does not exist')

In [ ]:
name_re = re.compile(r'(Nome do Aluno:) (.*) (RA)')
ra_re = re.compile(r'(RA:) (.*) (Ano Letivo:)')
#turma_re = re.compile(r'(Turma:) (.*) (ANUAL)')

In [ ]:


directory = '2_ANO'

if directory == '1_ANO':
    
    classes = ['1A', '1B', '1C', '1D', '1E', '1F', '1G', '1H', '1I', '1J', '1K']
else:
    classes = ['2A', '2B', '2C', '2D', '2E', '2F', '2G', '2H', '2I', '2J', '2K', '2L']

grades = {}
names = {}
class_ = {}
            

for c in classes[0:]:
    
    turma = c
    files = ['{directory}/{classes}.pdf'.format(directory = directory, classes=c)]
    print(files)

    for file in files:
        with pdfplumber.open(file) as pdf:

            
            
            for page in pdf.pages[0:]:

                tables = page.find_tables()

                text = page.extract_text()

                for line in text.split('\n'):
                    name = name_re.search(line)
                    if name:
                        name_A = name.group(2)

                    ra = ra_re.search(line)

                    if ra:
                        ra_A = str(ra.group(2))



                #print(name_A, ra_A, turma_A)

                for tables_num in range (0,(len(tables)-1)):
                    table = tables[tables_num].extract(x_tolerance = 5)

                    df = pd.DataFrame(table, columns=table[0])


                    try:
                        df1 = df.loc[:, ['Disciplina', '1º Bimestre', '2º Bimestre', '3º Bimestre', '4º Bimestre']] \
                                .replace({'ET': None
                                          , 'ES': None
                                          , 'EP': None
                                          , '-':None})
                        columnNames = ['Materia', 'BIM_1','BIM_2', 'BIM_3', 'BIM_4']
                        df1.columns = columnNames

                        df2 = df1[1:][1:]
                        df2 = df2.drop(index=df2[df2['Materia'].isin(['TECNOLOGIA E INOVACAO'
                                                                      , 'DIGITAL INFLUENCER'
                                                                      , 'INFLUÊNCIA DIGITAL'
                                                                      , 'FOTOGRAFIA'
                                                                      , 'INFLUENCIADOR DIGITAL'
                                                                      , 'PROFISSÕES'
                                                                      , 'PROJETO DE VIDA'
                                                                      ,'MANIFESTAÇÕES ARTÍSTICAS'
                                                                      #, 'EDUCACAO FISICA'
                                                                      , 'EDUCACAO FISICA - EXP'
                                                                      , 'EDUCACAO FISICA IF - EXP'
                                                                      , 'DEBATES'
                                                                      , 'PROJETO DE VIDA - EXP','GASTRONOMIA'
                                                                      , 'ARTES CÊNICAS'
                                                                      , 'MÚSICA E SOCIEDADE - 2ª A'
                                                                      , 'ORIENTACAO DE ESTUDOS - EXP'
                                                                      , 'ORIENTACAO DE ESTUDOS'
                                                                      , 'TECNOLOGIA E INOVACAO - EXP'
                                                                      , '2ªA - HISTÓRIA E CINEMA'
                                                                      , '2º H - FOTOGRAFIA NA PERSPECTIVA \nCIENTÍFICA'
                                                                      , '2º I- FOTOGRAFIA NA PERSPECTIVA \nCIENTÍFICA'
                                                                      , '2º H - FOTOGRAFIA NA \nPERSPECTIVA CIENTÍFICA'
                                                                      , '2º G - FOTOGRAFIA NA PERSPECTIVA \nCIENTÍFICA'
                                                                      , '2ªB - HISTÓRIA E CINEMA'
                                                                      , 'EMPREENDEDORISMO'
                                                                      , 'EDUCAÇÃO MIDIÁTICA'
                                                                      , 'CRIARTE'
                                                                      , '1K      CRIARTE'
                                                                      , '1F CRIARTE'
                                                                      , '1F     CRIARTE'
                                                                     , '2F CRIARTE'])].index)

                        if ra_A not in grades:
                            grades[ra_A] = []
                        
                        # append some value 
                        grades[ra_A].append(df2)
                        names[ra_A] = name_A
                        
                        if ra_A not in class_:
                            class_[ra_A] = []
                            class_[ra_A].append(c)
                            
                        else:
                            if c in class_[ra_A]:
                                True
                            else:
                                class_[ra_A].append(c)


                    except KeyError:
                        True      


            
            

In [ ]:
#ra = 'ra_number'

#student = "Nome: {name}, RA: {ra}, Turma: {turma}".format(name = names[ra], ra = ra, turma = class_[ra])
##
#all_students_grades = copy.copy(grades)

#print(all_students_grades['ra_number'])
#print(multiple_classes(all_students_grades['ra_number']))
#print(student)



In [ ]:
#This function deals with students that had moved to a different class along the year

def multiple_classes(x):
    
    size = len(x)
    
     
    #Assigning the grades of two different classes
    
    if size > 1:
        table = x[0]
        
        for num in range(1,size):
            
            table = pd.concat([table, x[num]], axis=0)                    
            

        
    else:
        table = x[0]
        
    #Checking how many bimonthly cycle the student has
    column_names = table.columns
    
    #convertendo as colunas com as notas em números
    for column_name in column_names[1:]:
        table[column_name] = pd.to_numeric(table[column_name], errors='coerce')


    final_grade = table.groupby(['Materia'], as_index=False)['BIM_1', "BIM_2", 'BIM_3', 'BIM_4'].max(min_count=1)

        
    return final_grade    
        
        

In [ ]:
#listing all students
import copy
import statistics

mean_grades = {}

all_students_grades = copy.copy(grades)

for key in all_students_grades.keys():
    
    student_total_classes = len(grades[key])
    
    if student_total_classes >=1:
        data = multiple_classes(all_students_grades[key])
 
        # Calculate the mean of the data
        
        df = pd.DataFrame(data)
        
        grade_mean = df.mean(axis = None, skipna=True).mean(skipna=True)
        
        mean_grades[key] = grade_mean
        
        #pint(all_students_grades[key])
        #result = '\n'.join(f'{key}: {value}' for key, value in all_students_grades.items())
        #print(result)
        #display(df)

In [ ]:
#ra = '000110356942-9/SP'

#student = "Nome: {name}, RA: {ra}, Média = {mean}, Turma: {turma}".format(name = names[ra], ra = ra, turma = class_[ra], mean = mean_grades[ra])
#print(student)



In [ ]:
# Create an empty DataFrame with columns A, B, and C
final_table = pd.DataFrame(columns=['Nome', 'RA', 'Turma', 'Média'])


for ra in names.keys():
    final_table = final_table.append({'Nome': names[ra], 'RA': ra, 'Turma': class_[ra], 'Média': mean_grades[ra]}, ignore_index=True)

ft_treated = final_table.sort_values(by='Média', ascending=False)

ft_treated['Média'] = ft_treated['Média'].apply(lambda x: str(x))
ft_treated['Média'] = ft_treated['Média'].apply(lambda x: x.replace('.', ','))

    # Print the resulting DataFrame
display(ft_treated)

In [ ]:
ft_treated.to_csv('medias_cdt_2022_sem2_{ano}.csv'.format(ano=directory), sep='\t')


In [ ]:
for ras in all_students_grades.keys():
    student = "Nome: {name}, RA: {ra}, Média = {mean}, Turma: {turma}".format(name = names[ras], ra = ras, turma = class_[ras], mean = mean_grades[ras])
    print(student)


In [ ]:


x = 0


all_info_table = pd.DataFrame(columns=['Materia', 'BIM_1', 'BIM_2', 'BIM_3', 'BIM_4', 'RA', 'Nome', 'Turma'])

for key in all_students_grades.keys():
      
    d = multiple_classes(all_students_grades[key])
    d['RA'] = key
    d['Nome'] = names[key] 
    d['Turma'] = str(class_[key])
    
    all_bim = ['BIM_1', 'BIM_2', 'BIM_3', 'BIM_4']
    
    for bim in all_bim:  
        d[bim] = d[bim].apply(lambda x: str(x))
        d[bim] = d[bim].apply(lambda x: x.replace('.', ','))
    
    d_ = pd.DataFrame(d)
    
    
    all_info_table = all_info_table.append(d_, ignore_index=False)

all_info_table = all_info_table[['Nome', 'Turma', 'Materia', 'BIM_1', 'BIM_2', 'BIM_3', 'BIM_4']]

all_info_table.to_csv('medias_cdt_2022_sem2_{ano}_all_grades.csv'.format(ano=directory), sep='\t')



In [ ]:
display(all_info_table.sort_values(by='Nome', ascending=True))